In [1]:
import sys
import os

sys.path.append('../')

In [2]:
import joblib
from paper_markdown_text_classifier import *

model = joblib.load("TextClassifie-full-final.pkl")

# 随机测试方法

In [5]:
import os
from tqdm import tqdm
from pathlib import Path
import random
import datasets

counterexamples = list(Path("../data/docx").glob("**/*.doc*"))

def generate_random_tests(total=100):
    random_numbers_list = random.sample(range(0, len(counterexamples) + 1), total)
    
    test_dataset = {"text":[], "file_path":[]}
    for random_number in tqdm(random_numbers_list):
        file_path = counterexamples[random_number]
      
        _, ext = os.path.splitext(file_path)
        try:
            text = extract_text(str(file_path), ext)
        except:
            continue

        test_dataset["text"].append(text)
        test_dataset["file_path"].append(str(file_path))

    test_dataset = datasets.Dataset.from_dict(test_dataset)
    test_dataset = test_dataset.map(dataset_map_pre_process)
    return test_dataset

In [7]:
import os
from tqdm import tqdm
from pathlib import Path
import sys
import os
import random
sys.path.append('../')

from paper_markdown_text_classifier import extract_text


counterexamples = list(Path("../data/docx").glob("**/*.doc*"))

def generate_random_test_dataset(total=100):
    random_numbers_list = random.sample(range(0, len(counterexamples)), total)
    
    test_dataset = {"text":[], "file_path":[]}
    for random_number in tqdm(random_numbers_list):
        file_path = counterexamples[random_number]
      
        _, ext = os.path.splitext(file_path)
        try:
            text = extract_text(str(file_path), ext)
        except:
            continue

        test_dataset["text"].append(text)
        test_dataset["file_path"].append(str(file_path))

    test_dataset = datasets.Dataset.from_dict(test_dataset)
    test_dataset = test_dataset.map(dataset_map_pre_process)
    return test_dataset

In [8]:
def move_file_to_negative(file_path):
    file_name = file_path[file_path.rfind("/")+1:]
    shutil.copy(file_path, os.path.join("negative_file/", file_name))

def move_file_to_positive(file_path):
    file_name = file_path[file_path.rfind("/")+1:]
    shutil.copy(file_path, os.path.join("positive_file/", file_name))

In [16]:
error_file = []

def start_test(test_dataset, model):
    global error_file
    
    predictions, positive_probabilities = get_predict_with_threshold(model, test_dataset["text"],0.5)
    
    dataset_row_list = [row for row in test_dataset]
    zipped = list(zip(dataset_row_list, positive_probabilities))

    sorted_zipped = sorted(zipped, key=lambda x: float(x[1]))

    for index, (row, prediction) in enumerate(sorted_zipped):
        scope = '{:.2f}'.format(prediction)
        if scope not in  ["0.00", "1.00"]:
            print(index, " --- ",scope)
            file_path = row['file_path'].rsplit("/", 2)
            print("/".join(file_path[-2:]))
            print()

            
    return sorted_zipped

In [17]:
test_dataset = None
sorted_zipped = None

for _ in range(1):
    test_dataset = generate_random_test_dataset()
    sorted_zipped = start_test(test_dataset, model)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.45it/s]


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

72  ---  0.01
200套WORD简历模板/133.新生命力_表格式_适用于IT、工程_2页式_无内容_有封面.doc

73  ---  0.01
KPI说明书/朝阳电子人力资源项目—02-3培训计划达成率月周报表填明说明.doc

74  ---  0.02
初次访谈/初次访谈日程安排与需配合事项.doc

75  ---  0.03
调研问卷模板/汉普-长沙卷烟-海信部门调研说明.doc

76  ---  0.04
采購部200502/华盈恒信—金德精密—改善計劃(05年02月)採購總額降價.doc

77  ---  0.05
制度/市场部员工关键事件评分细则.doc

78  ---  0.06
03绩效考核细则/2003年经营管理工作考核细则.doc

79  ---  0.34
备份/测试.doc

80  ---  0.90
讲义/jczs_qh_mst_jy1031.doc

81  ---  0.98
32、通讯、广告IT可行性报告/【公益知识库zscc.club】汽车修理厂可行性报告.doc



In [18]:
view_index = 80

print(sorted_zipped[view_index][0]["file_path"])
print()
for line in sorted_zipped[view_index][0]["text"].splitlines():
    print(line)

../data/docx/202307/⛵【知识船舱】60TB内部学习资源(大合集)/🌠 科学馆【1.24TB】/💊 中医宝典【877.78GB】/中医内科主治医师【13.30GB】/中医内科主治医师-基础知识（2021）【全】【5.03GB】/讲义/jczs_qh_mst_jy1031.doc

　　精、气、血、津液、神
细目       内容                                                 
精概念    禀受于父母的生命物质与后天水谷精微相融合而形成的一种 
          精华物质                                             
  生成    以先天之精为本，并得到后天之精的不断充养而生成       
  贮藏    主要藏于肾中（肾气封藏）                             
  施泄    濡养脏腑，化气以推动和调控各脏腑机能；化为生殖之精   
  生理功能繁衍生命、营养周身、化血、化气、化神                 
  分类    先天之精、后天之精、脏腑之精、生殖之精               
气概念    是人体内活力很强、运行不息的极精微物质               
  生成    由精所化生，并与肺吸入的自然界清气相融合而成         
          肾为生气之根、脾胃为生气之源、肺为生气之主           
  运动    气的运动—气机，气运动的基本形式—升、降、出、入       
  气化    由于气的运动而产生的各种变化—气化                    
          　　气化即是体内物质新陈代谢的过程                   
  气机与气气的运动是产生气化过程的根本。气的升降出入运动及气的 
  化的关系阴阳双方的相互作用又是气化过程发生和赖以进行的前提与 
          条件                                                 
  气的功能推动与调控、温煦与凉润、防御、固摄、中介             
气元气    最基本最重要之气                           

In [593]:
move_file_to_negative(sorted_zipped[90][0]["file_path"])

In [638]:
move_file_to_positive(sorted_zipped[82][0]["file_path"])